### I. Extract data via polygon.io API and explore contents

In [1]:
# install package
#pip install polygon-api-client

In [2]:
# Import libraries
import polygon
#from polygon import RESTClient
from collections import Counter
import inspect
import pandas as pd
import numpy as np

# Set the global display format for numbers with commas
pd.options.display.float_format = '{:,.0f}'.format

In [4]:
# Get free API key from polygon to access data
my_API_key = 'LIAPhgx5dHDw_pZ0bKwXMBmT9wnjPHGd'
client = polygon.RESTClient(my_API_key)

In [5]:
# Get an example ticker to inspect data
example_ticker = 'PWSC'

In [6]:
# Call data from API
data = client.vx.list_stock_financials(ticker = example_ticker)

In [7]:
# Check data type
type(data)

generator

**Observations**:

Interesting that query output is a generator, which is an iterable object that produces values one at a time. Thus, in otder to inspect the contents, we can put it in a list and iterate through each component

In [8]:
# Put extracted data in a list
data_list = list(data)

In [9]:
# Get a count of values
component_count = len(data_list)
print(f"Number of entries in extracted generator: {component_count}" )

Number of entries in extracted generator: 10


In [10]:
# Print out content
for item in data_list:
    print(item)

StockFinancial(cik='0001835681', company_name='POWERSCHOOL HOLDINGS, INC.', end_date='2023-09-30', filing_date='2023-11-09', financials=Financials(balance_sheet={'noncurrent_liabilities': DataPoint(formula=None, label='Noncurrent Liabilities', order=800, unit='USD', value=1507808000.0, xpath=None), 'equity_attributable_to_parent': DataPoint(formula=None, label='Equity Attributable To Parent', order=1600, unit='USD', value=1303319000.0, xpath=None), 'current_liabilities': DataPoint(formula=None, label='Current Liabilities', order=700, unit='USD', value=542507000.0, xpath=None), 'liabilities': DataPoint(formula=None, label='Liabilities', order=600, unit='USD', value=2050315000.0, xpath=None), 'other_noncurrent_liabilities': DataPoint(formula=None, label='Other Non-current Liabilities', order=820, unit='USD', value=669882000.0, xpath=None), 'other_noncurrent_assets': DataPoint(formula=None, label='Other Non-current Assets', order=350, unit='USD', value=2652743000.0, xpath=None), 'noncurre

**Observations**
Each item is an entry of financial data for different end_date and filing_date

In [11]:
# Take a look at structure of the first item (second item if counting from index 0)
current_row = data_list[7]
vars(current_row).keys()

dict_keys(['cik', 'company_name', 'end_date', 'filing_date', 'financials', 'fiscal_period', 'fiscal_year', 'source_filing_file_url', 'source_filing_url', 'start_date'])

**Observations:** 
Main data fields include 'cik', 'company_name', 'end_date', 'filing_date', 'financials', 'fiscal_period',
'fiscal_year', 'from_dict', 'source_filing_file_url', 'source_filing_url', 'start_date'

**Extract Basic Stock Ticker Data**

In [12]:
# Get a list of attribute
attribute_keys = vars(current_row).keys()

# Initialize an empty dictionary to extract its value
attribute_dict = {}

# Extract attribute values to put in the new dictionary
for attribute_name in attribute_keys:
    attribute_value = getattr(current_row, attribute_name)
    attribute_dict[attribute_name] = attribute_value
    
# Get a smaller dictionary for basic info
meta_data_dict = {key: value for key, value in attribute_dict.items() if key != 'financials'}
meta_data_df = pd.DataFrame(meta_data_dict, index = [0])

# Define the desired column order
desired_col_order = ['cik','company_name', 'start_date', 'end_date', 'filing_date', 'fiscal_year', 'fiscal_period', 
                     'source_filing_file_url', 'source_filing_url']
meta_data_df = meta_data_df[desired_col_order]

# Display output dataframe
display(meta_data_df)

,cik,company_name,start_date,end_date,filing_date,fiscal_year,fiscal_period,source_filing_file_url,source_filing_url
0,0001835681,"POWERSCHOOL HOLDINGS, INC.",2022-01-01,2022-03-30,None,2022,Q1,None,None


In [13]:
# Look at attribute keys of Financials data
financial_keys = vars(attribute_dict['financials']).keys()
display(financial_keys)

dict_keys(['balance_sheet', 'cash_flow_statement', 'comprehensive_income', 'income_statement'])

In [14]:
# Iterate through each key in financial_keys to inspect the data type of each
for key in financial_keys:
    financial_statement_dict = getattr(attribute_dict['financials'], key)
    print(f"\nType of {key}: ")
    print(type(financial_statement_dict))


Type of balance_sheet: 
<class 'dict'>

Type of cash_flow_statement: 
<class 'NoneType'>

Type of comprehensive_income: 
<class 'polygon.rest.models.financials.ComprehensiveIncome'>

Type of income_statement: 
<class 'polygon.rest.models.financials.IncomeStatement'>


**Extract Balance Sheet Data**

In [15]:
# Look at content of Balance Sheet
print(current_row.financials.balance_sheet)

{'equity': DataPoint(formula=None, label='Equity', order=1400, unit='USD', value=1719058000.0, xpath=None), 'equity_attributable_to_parent': DataPoint(formula=None, label='Equity Attributable To Parent', order=1600, unit='USD', value=1727003000.0, xpath=None), 'current_liabilities': DataPoint(formula=None, label='Current Liabilities', order=700, unit='USD', value=-1719058000.0, xpath=None), 'equity_attributable_to_noncontrolling_interest': DataPoint(formula=None, label='Equity Attributable To Noncontrolling Interest', order=1500, unit='USD', value=-7945000.0, xpath=None), 'liabilities': DataPoint(formula=None, label='Liabilities', order=600, unit='USD', value=-1719058000.0, xpath=None), 'noncurrent_liabilities': DataPoint(formula=None, label='Noncurrent Liabilities', order=800, unit='USD', value=0, xpath=None)}


In [16]:
# Look at content of Balance Sheet keys
print(current_row.financials.balance_sheet.keys())

dict_keys(['equity', 'equity_attributable_to_parent', 'current_liabilities', 'equity_attributable_to_noncontrolling_interest', 'liabilities', 'noncurrent_liabilities'])


In [17]:
type(current_row.financials.balance_sheet)

dict

In [18]:
# Put extracted data into a dictionary
balance_sheet_dict = current_row.financials.balance_sheet

# Initialize empty dictionary to store data
data = {}

# Iterate through the dictionary
for key, datapoint in balance_sheet_dict.items():
    data[key] = datapoint.value

# Create a DataFrame
balance_sheet_df = pd.DataFrame(data,  index=[0])

# Print the resulting DataFrame
display(balance_sheet_df)

,equity,equity_attributable_to_parent,current_liabilities,equity_attributable_to_noncontrolling_interest,liabilities,noncurrent_liabilities
0,"1,719,058,000","1,727,003,000","-1,719,058,000","-7,945,000","-1,719,058,000",0


**Extract Cash Flow Data**

In [19]:
print(current_row.financials.cash_flow_statement)

None


In [20]:
type(current_row.financials.cash_flow_statement)

NoneType

In [21]:
# Check if cash_flow_statement is not None, and return nothing if it is
if current_row.financials.cash_flow_statement is not None:
    cashflow_dict = vars(current_row.financials.cash_flow_statement)

    # Initialize empty dictionary to store data
    data = {}

    # Iterate through the dictionary
    for key, datapoint in cashflow_dict.items():
        data[key] = datapoint.value

    # Create a DataFrame
    cashflow_df = pd.DataFrame(data, index=[0])

    # Print the resulting DataFrame
    display(cashflow_df)

else:
    print("cash_flow_statement is None, nothing to display.")
    cashflow_df = pd.DataFrame()

cash_flow_statement is None, nothing to display.


**Extract Comprehensive Income Statement Data**

In [22]:
print(current_row.financials.comprehensive_income)

ComprehensiveIncome(comprehensive_income_loss=ComprehensiveIncomeLoss(formula=None, label='Comprehensive Income/Loss', order=100, unit='USD', value=-14120000.0, xpath=None), comprehensive_income_loss_attributable_to_parent=ComprehensiveIncomeLossAttributableToParent(formula=None, label='Comprehensive Income/Loss Attributable To Parent', order=300, unit='USD', value=-14120000.0, xpath=None), other_comprehensive_income_loss=OtherComprehensiveIncomeLoss(formula=None, label='Other Comprehensive Income/Loss', order=400, unit='USD', value=0, xpath=None))


In [23]:
# Put data in a dictionary
comprehensive_income_dict = vars(current_row.financials.comprehensive_income)

# Initialize empty dictionary to store data
data = {}

# Iterate through the dictionary
for key, datapoint in comprehensive_income_dict.items():
    data[key] = datapoint.value

# Create a DataFrame
comprehensive_income_df = pd.DataFrame(data, index=[0])

# Print the resulting DataFrame
display(comprehensive_income_df)

,comprehensive_income_loss,comprehensive_income_loss_attributable_to_parent,other_comprehensive_income_loss
0,"-14,120,000","-14,120,000",0


**Extract Income Statement Data**

In [24]:
print(current_row.financials.income_statement)

IncomeStatement(basic_earnings_per_share=BasicEarningsPerShare(formula=None, label='Basic Earnings Per Share', order=4200, unit='USD / shares', value=-0.08, xpath=None), cost_of_revenue=CostOfRevenue(formula=None, label='Cost Of Revenue', order=300, unit='USD', value=67976000.0, xpath=None), gross_profit=GrossProfit(formula=None, label='Gross Profit', order=800, unit='USD', value=81617000.0, xpath=None), operating_expenses=OperatingExpenses(formula=None, label='Operating Expenses', order=1000, unit='USD', value=84254000.0, xpath=None), revenues=Revenues(formula=None, label='Revenues', order=100, unit='USD', value=149593000.0, xpath=None))


In [25]:
# Put data in a dictionary
income_statement_dict = vars(current_row.financials.income_statement)

# Initialize empty dictionary to store data
data = {}

# Iterate through the dictionary
for key, datapoint in income_statement_dict.items():
    data[key] = datapoint.value

# Create a DataFrame
income_statement_df = pd.DataFrame(data, index=[0])

# Print the resulting DataFrame
display(income_statement_df)

,basic_earnings_per_share,cost_of_revenue,gross_profit,operating_expenses,revenues
0,-0,"67,976,000","81,617,000","84,254,000","149,593,000"


**Put all df together**

In [26]:
# Concatenate the DataFrames column-wise
consolidated_df = pd.concat([meta_data_df, balance_sheet_df, cashflow_df, comprehensive_income_df, income_statement_df], axis=1)
consolidated_df.set_index('cik', inplace=True)

# Print the resulting DataFrame
display(consolidated_df)

,company_name,start_date,end_date,filing_date,fiscal_year,fiscal_period,source_filing_file_url,source_filing_url,equity,equity_attributable_to_parent,...,liabilities,noncurrent_liabilities,comprehensive_income_loss,comprehensive_income_loss_attributable_to_parent,other_comprehensive_income_loss,basic_earnings_per_share,cost_of_revenue,gross_profit,operating_expenses,revenues
cik,,,,,,,,,,,,,,,,,,,,,
0001835681,"POWERSCHOOL HOLDINGS, INC.",2022-01-01,2022-03-30,None,2022,Q1,None,None,"1,719,058,000","1,727,003,000",...,"-1,719,058,000",0,"-14,120,000","-14,120,000",0,-0,"67,976,000","81,617,000","84,254,000","149,593,000"


### II. Run a loop to all index extract all data for sample stocks

In [27]:
# Initialize an empty list to store DataFrames from each iteration
result_df = pd.DataFrame()

# Get a count of entries in the object returned by API call
component_count = len(data_list)

for i in range(component_count):
    # Get a list of attribute
    attribute_keys = vars(data_list[i]).keys()

    # Initialize an empty dictionary to extract its value
    attribute_dict = {}

    # Extract attribute values to put in the new dictionary
    for attribute_name in attribute_keys:
        attribute_value = getattr(data_list[i], attribute_name)
        attribute_dict[attribute_name] = attribute_value

    # Get a smaller dictionary for basic info
    meta_data_dict = {key: value for key, value in attribute_dict.items() if key != 'financials'}
    meta_data_df = pd.DataFrame(meta_data_dict, index = [0])

    # Define the desired column order
    desired_col_order = ['cik','company_name', 'start_date', 'end_date', 'filing_date', 'fiscal_year', 'fiscal_period', 
                         'source_filing_file_url', 'source_filing_url']
    meta_data_df = meta_data_df[desired_col_order]
    
    # Get a list of keys from attribute 'financials'
    financial_keys = vars(attribute_dict['financials']).keys()
    
    # Put financials object into a variable
    financial_object = attribute_dict['financials']
    
    # Create an empty list to store DataFrames to consolidate all df extracted from BS, CF, CIS, IS
    financial_statement_dfs = []

    # Loop through each key in financials attribute of each data_list[index] row
    for eachkey in financial_keys:

        # Extract value from financial object - could be a dictionary or polygon object
        financial_statement_obj = getattr(financial_object, eachkey)

        # If financial_statement_obj type is a dictionary, then do nothing, 
        # but if the type is a polygon object, then convert it to a dictionary using vars()
        if isinstance(financial_statement_obj, dict):
            # Put in dictionary with a standardized name
            financial_statement_dict = financial_statement_obj
        
        elif isinstance(financial_statement_obj, (polygon.rest.models.CashFlowStatement, 
                                                  polygon.rest.models.ComprehensiveIncome, 
                                                  polygon.rest.models.IncomeStatement)):
            
            financial_statement_dict = vars(financial_statement_obj)
        
        else:
            # Put the rest to empty dictionary
            financial_statement_dict = {}

        # Initialize an empty dictionary to store data
        extracted_data = {}

        # Iterate through the dictionary
        for key, datapoint in financial_statement_dict.items():
            extracted_data[key] = datapoint.value

        # Create a DataFrame to put extracted data dictionary into
        financial_statement_df = pd.DataFrame(extracted_data, index=[0])

        # Append the DataFrame to the list
        financial_statement_dfs.append(financial_statement_df)

    # Concatenate all DataFrames in the list column-wise
    combined_df = pd.concat(financial_statement_dfs, axis=1)
    
    # Add meta_data columns to combined_df
    combined_df = pd.concat([meta_data_df, combined_df], axis=1)
    
    # Display the number of columns and their names in the DataFrame
    #print(f"DataFrame {i+1} - Number of columns: {combined_df.shape[1]}")
    
    # Check for duplicate columns in the combined_df
    #duplicate_columns = [col for col, count in Counter(combined_df.columns).items() if count > 1]
    #if duplicate_columns:
    #    print(f"DataFrame {i+1} - Duplicate columns in combined DataFrame: {', '.join(duplicate_columns)}")
    
    # Change empty cells to np.NaN
    combined_df = combined_df.replace('', np.nan)
    
    # Change empty cells to np.NaN
    combined_df = combined_df.replace('None', np.nan)
        
    # Append the resulting DataFrame to the list for all entries in query
    result_df = pd.concat([result_df,combined_df], axis=0, ignore_index=True)

# Display the resulting combined DataFrame
display(result_df)

,cik,company_name,start_date,end_date,filing_date,fiscal_year,fiscal_period,source_filing_file_url,source_filing_url,noncurrent_liabilities,...,net_cash_flow,net_cash_flow_from_financing_activities,comprehensive_income_loss,comprehensive_income_loss_attributable_to_parent,other_comprehensive_income_loss,basic_earnings_per_share,cost_of_revenue,gross_profit,operating_expenses,revenues
0,0001835681,"POWERSCHOOL HOLDINGS, INC.",2023-07-01,2023-09-30,2023-11-09,2023,Q3,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,"1,507,808,000",...,"294,437,000","96,635,000","-647,000","-614,000","659,000",0,"71,850,000","110,314,000","98,653,000","182,164,000"
1,0001835681,"POWERSCHOOL HOLDINGS, INC.",2023-04-01,2023-06-30,2023-08-08,2023,Q2,https://api.polygon.io/v1/reference/sec/filing...,https://api.polygon.io/v1/reference/sec/filing...,0,...,"-35,880,000","7,922,000","-4,295,000","-3,178,000",0,-0,"69,029,000","104,868,000","94,755,000","173,897,000"
2,0001835681,"POWERSCHOOL HOLDINGS, INC.",2023-01-01,2023-03-31,2023-05-05,2023,Q1,https://api.polygon.io/v1/reference/sec/filing...,https://api.polygon.io/v1/reference/sec/filing...,0,...,"-73,208,000","-3,222,000","-14,813,000","-11,781,000",0,-0,"69,489,000","89,965,000","90,750,000","159,454,000"
3,0001835681,"POWERSCHOOL HOLDINGS, INC.",2022-10-01,2022-12-31,None,2022,Q4,None,None,"1,435,754,000",...,"28,598,000","-4,300,000","-1,784,000","-1,752,000","1,462,000",-0,"66,600,000","94,466,000","88,107,000","161,066,000"
4,0001835681,"POWERSCHOOL HOLDINGS, INC.",2022-01-01,2022-12-31,2023-02-24,2022,FY,https://api.polygon.io/v1/reference/sec/filing...,https://api.polygon.io/v1/reference/sec/filing...,"1,435,754,000",...,"50,990,000","-19,232,000","-22,693,000","-22,311,000","5,048,000",-0,"272,347,000","358,336,000","352,432,000","630,683,000"
5,0001835681,"POWERSCHOOL HOLDINGS, INC.",2022-07-01,2022-09-30,2022-11-08,2022,Q3,https://api.polygon.io/v1/reference/sec/filing...,https://api.polygon.io/v1/reference/sec/filing...,"1,438,697,000",...,"93,428,000","-80,762,000","-3,269,000","-3,120,000","648,000",-0,"69,787,000","92,647,000","89,317,000","162,434,000"
6,0001835681,"POWERSCHOOL HOLDINGS, INC.",2022-04-01,2022-06-30,2022-08-08,2022,Q2,https://api.polygon.io/v1/reference/sec/filing...,https://api.polygon.io/v1/reference/sec/filing...,0,...,NaN,NaN,"-6,458,000","-4,250,000",0,-0,"67,985,000","89,606,000","90,752,000","157,591,000"
7,0001835681,"POWERSCHOOL HOLDINGS, INC.",2022-01-01,2022-03-30,None,2022,Q1,None,None,0,...,NaN,NaN,"-14,120,000","-14,120,000",0,-0,"67,976,000","81,617,000","84,254,000","149,593,000"
8,0001835681,"POWERSCHOOL HOLDINGS, INC.",2021-01-01,2021-12-31,2022-03-24,2021,FY,https://api.polygon.io/v1/reference/sec/filing...,https://api.polygon.io/v1/reference/sec/filing...,"1,448,082,000",...,"33,745,000","264,699,000","-34,323,000","-34,268,000","8,742,000",-0,"240,858,000","317,740,000","312,024,000","558,598,000"
9,0001835681,"POWERSCHOOL HOLDINGS, INC.",2021-07-01,2021-09-30,2021-11-10,2021,Q3,https://api.polygon.io/v1/reference/sec/filing...,https://api.polygon.io/v1/reference/sec/filing...,"1,452,846,000",...,NaN,NaN,"-19,712,000","-19,701,000","5,416,000",-0,"63,332,000","85,620,000","88,074,000","148,952,000"


In [28]:
# Look at the result dataframe structure
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 38 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   cik                                               10 non-null     object 
 1   company_name                                      10 non-null     object 
 2   start_date                                        10 non-null     object 
 3   end_date                                          10 non-null     object 
 4   filing_date                                       8 non-null      object 
 5   fiscal_year                                       10 non-null     object 
 6   fiscal_period                                     10 non-null     object 
 7   source_filing_file_url                            8 non-null      object 
 8   source_filing_url                                 8 non-null      object 
 9   noncurrent_liabilities  

In [29]:
# Get the final dataframe that only contains rows with all columns populated
# Drop columns with any missing values (NaN)
result_df_filtered = result_df.dropna(axis=1, how='any')
result_df_filtered

,cik,company_name,start_date,end_date,fiscal_year,fiscal_period,noncurrent_liabilities,equity_attributable_to_parent,current_liabilities,liabilities,equity_attributable_to_noncontrolling_interest,equity,comprehensive_income_loss,comprehensive_income_loss_attributable_to_parent,other_comprehensive_income_loss,basic_earnings_per_share,cost_of_revenue,gross_profit,operating_expenses,revenues
0,0001835681,"POWERSCHOOL HOLDINGS, INC.",2023-07-01,2023-09-30,2023,Q3,"1,507,808,000","1,303,319,000","542,507,000","2,050,315,000","457,048,000","1,760,367,000","-647,000","-614,000","659,000",0,"71,850,000","110,314,000","98,653,000","182,164,000"
1,0001835681,"POWERSCHOOL HOLDINGS, INC.",2023-04-01,2023-06-30,2023,Q2,0,"1,746,030,000","312,085,000","1,715,133,000","456,734,000","1,746,030,000","-4,295,000","-3,178,000",0,-0,"69,029,000","104,868,000","94,755,000","173,897,000"
2,0001835681,"POWERSCHOOL HOLDINGS, INC.",2023-01-01,2023-03-31,2023,Q1,0,"1,276,514,000","348,575,000","1,757,145,000","455,447,000","1,731,961,000","-14,813,000","-11,781,000",0,-0,"69,489,000","89,965,000","90,750,000","159,454,000"
3,0001835681,"POWERSCHOOL HOLDINGS, INC.",2022-10-01,2022-12-31,2022,Q4,"1,435,754,000","1,248,667,000","413,697,000","1,849,451,000","485,277,000","1,733,944,000","-1,784,000","-1,752,000","1,462,000",-0,"66,600,000","94,466,000","88,107,000","161,066,000"
4,0001835681,"POWERSCHOOL HOLDINGS, INC.",2022-01-01,2022-12-31,2022,FY,"1,435,754,000","1,248,667,000","413,697,000","1,849,451,000","485,277,000","1,733,944,000","-22,693,000","-22,311,000","5,048,000",-0,"272,347,000","358,336,000","352,432,000","630,683,000"
5,0001835681,"POWERSCHOOL HOLDINGS, INC.",2022-07-01,2022-09-30,2022,Q3,"1,438,697,000","1,242,095,000","457,687,000","1,896,384,000","485,568,000","1,727,663,000","-3,269,000","-3,120,000","648,000",-0,"69,787,000","92,647,000","89,317,000","162,434,000"
6,0001835681,"POWERSCHOOL HOLDINGS, INC.",2022-04-01,2022-06-30,2022,Q2,0,"1,727,003,000","330,197,000","1,775,458,000","488,902,000","1,719,058,000","-6,458,000","-4,250,000",0,-0,"67,985,000","89,606,000","90,752,000","157,591,000"
7,0001835681,"POWERSCHOOL HOLDINGS, INC.",2022-01-01,2022-03-30,2022,Q1,0,"1,727,003,000","-1,719,058,000","-1,719,058,000","-7,945,000","1,719,058,000","-14,120,000","-14,120,000",0,-0,"67,976,000","81,617,000","84,254,000","149,593,000"
8,0001835681,"POWERSCHOOL HOLDINGS, INC.",2021-01-01,2021-12-31,2021,FY,"1,448,082,000","1,234,745,000","385,642,000","1,833,724,000","488,213,000","1,722,958,000","-34,323,000","-34,268,000","8,742,000",-0,"240,858,000","317,740,000","312,024,000","558,598,000"
9,0001835681,"POWERSCHOOL HOLDINGS, INC.",2021-07-01,2021-09-30,2021,Q3,"1,452,846,000","1,237,350,000","428,048,000","1,880,894,000","489,449,000","1,726,799,000","-19,712,000","-19,701,000","5,416,000",-0,"63,332,000","85,620,000","88,074,000","148,952,000"


### Observations: There are A lot of columns with missing value and list of financial statement line items available are significantly shorter than other data sources like yahoo finance or macrotrend. Thus, this vendor API is not helpful to my analysis of financial statements for a large number of stocks